In [5]:
!pip install dicto

  Running setup.py bdist_wheel for dicto ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/bb/15/ba/790a9ef4e2f517545f71681f0b467665821110a24c6db82cdd
Successfully built dicto


In [10]:
%matplotlib inline
import pickle
import dicto as do

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
with open("/data/train.p", "rb") as fd:
    train = pickle.load(fd)

with open("/data/test.p", "rb") as fd:
    test = pickle.load(fd)

In [17]:
def parse_function(image, label):

    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    return image, label


def train_preprocess(image, label):

    if label in [11, 12, 13, 15, 17, 18, 22, 26, 30, 35]:
            image = tf.image.random_flip_left_right(image)

    if label in [1, 5, 12, 15, 17]:
        image = tf.image.random_flip_up_down(image)

    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)

    # Make sure the image is still in [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label

params = do.Dicto(
    buffer_size = 34799,
    batch_size = 16,
    epochs = 10
)

def input_fn(images, labels, params, training):
    
    ds = tf.data.Dataset.from_tensor_slices((
        images,
        labels,
    ))

    ds = ds.map(parse_function, num_parallel_calls=4)
    
    if training:
        ds = ds.map(train_preprocess, num_parallel_calls=4)
        ds = ds.apply(tf.data.experimental.shuffle_and_repeat( 
        buffer_size = params.buffer_size,
        count = params.epochs,
        ))
        
    #ds  ds.map(lambda x, y: ({"images": x}, y))

    # ds = ds.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.int32)))
        
    ds = ds.batch(params.batch_size, drop_remainder=True)
    ds = ds.prefetch(buffer_size=2)

    
    return ds

ds = input_fn(train['features'], train['labels'], params, training=True)

train_input_fn = lambda : input_fn(train['features'].astype(np.float32), train['labels'].astype(np.int32), params, training=True)
eval_input_fn = lambda : input_fn(test['features'], test['labels'], params, training=False)


In [19]:
iter_ds = ds.make_one_shot_iterator()

In [20]:
elem_ops = iter_ds.get_next()

In [21]:
sess = tf.Session()

In [22]:
sess.run(elem_ops)

(array([[[[ 0.39064872,  0.31792414,  0.27117264],
          [ 0.94358993,  0.75658381,  0.72541612],
          [ 1.        ,  1.        ,  1.        ],
          ..., 
          [ 0.88380069,  0.90977377,  0.93574679],
          [ 0.66302216,  0.69418985,  0.73574674],
          [ 0.43388841,  0.44427761,  0.44947225]],
 
         [[ 0.42202127,  0.38046438,  0.32332364],
          [ 0.9592762 ,  0.87096781,  0.83460552],
          [ 1.        ,  1.        ,  1.        ],
          ..., 
          [ 0.87860608,  0.88380069,  0.93574679],
          [ 0.64448249,  0.64448249,  0.68084478],
          [ 0.36192468,  0.36711931,  0.35153547]],
 
         [[ 0.47300166,  0.40547168,  0.34833097],
          [ 0.96319777,  0.83852708,  0.80216479],
          [ 1.        ,  1.        ,  1.        ],
          ..., 
          [ 0.77165544,  0.79243386,  0.86515856],
          [ 0.44193643,  0.48868793,  0.48868793],
          [ 0.28359577,  0.30437422,  0.26281732]],
 
         ..., 
         [